# Практическая часть

# Import libs

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

## Read data

In [ ]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# Process features dataset

In [ ]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

In [ ]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [ ]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [ ]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [ ]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [ ]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [ ]:
# выше видим разброс по пользователям и товарам

In [ ]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


# Prefilter items

In [ ]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/home/aleks/rec sys/webinar_6/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001


# Make cold-start to warm-start

In [ ]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


In [ ]:
# Теперь warm-start по пользователям

# Init/train recommender

In [ ]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [ ]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(3)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67..."


In [ ]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [ ]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

## Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [ ]:
# N = Neighbors
N_PREDICT = 50 

In [ ]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=50))

CPU times: user 1min 36s, sys: 2min 53s, total: 4min 30s
Wall time: 45.4 s


In [ ]:
%%time
result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))
result_eval_matcher.head(5)

CPU times: user 14min 39s, sys: 19min 58s, total: 34min 38s
Wall time: 7min 46s


,user_id,actual,own_rec,sim_item_rec,als_rec,sim_user_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[839818, 1007512, 9297615, 5577022, 1132231, 9...","[1037332, 6533936, 856942, 1094924, 6534030, 8...","[1016785, 940770, 949257, 1135983, 918598, 994..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 985999, 880888, 81...","[916122, 866211, 5569230, 933835, 1090931, 931...","[9802756, 961889, 965772, 963542, 841287, 1063..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[6391541, 1052294, 891423, 936470, 1137010, 11...","[1139525, 846550, 1128539, 865528, 6514160, 84...","[891423, 821741, 944588, 1082627, 1137346, 108...","[7467039, 862535, 5566809, 998519, 906202, 112..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[13003092, 972416, 995598, 923600, 1138596, 10...","[948650, 5569845, 890739, 941361, 1074754, 110...","[1026118, 965267, 878996, 863632, 871611, 1084...","[7442505, 1113675, 948640, 907099, 1092669, 86..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[998519, 894360, 7147142, 9338009, 896666, 939...","[5585510, 1044078, 7147145, 12384779, 9707461,...","[912451, 1039627, 7142937, 1100140, 10285022, ...","[852015, 6552318, 8090550, 868888, 8069118, 80..."


In [ ]:
result_eval_matcher[['own_rec', 'sim_item_rec']].iloc[0]

own_rec         [856942, 9297615, 5577022, 877391, 9655212, 88...
sim_item_rec    [839818, 1007512, 9297615, 5577022, 1132231, 9...
Name: 0, dtype: object

In [ ]:
def rec_list_concat(*lists, res_len=N_PREDICT):
    res = []
    for i in range(len(lists[0])):
        for j in range(len(lists)):
            res.append(lists[j][i])
    return res[:res_len]


result_eval_matcher['als_and_own_rec'] = list(map(rec_list_concat, result_eval_matcher['als_rec'], result_eval_matcher['own_rec']))
result_eval_matcher['own_and_als_rec'] = list(map(rec_list_concat, result_eval_matcher['own_rec'], result_eval_matcher['als_rec']))
result_eval_matcher['own_and_sim_item_rec'] = list(map(rec_list_concat, result_eval_matcher['own_rec'],result_eval_matcher['sim_item_rec']))
result_eval_matcher['als_and_sim_item_rec'] = list(map(rec_list_concat, result_eval_matcher['own_rec'], result_eval_matcher['als_rec']))
result_eval_matcher.head()

,user_id,actual,own_rec,sim_item_rec,als_rec,sim_user_rec,als_and_own_rec,own_and_als_rec,own_and_sim_item_rec,als_and_sim_item_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[839818, 1007512, 9297615, 5577022, 1132231, 9...","[1037332, 6533936, 856942, 1094924, 6534030, 8...","[1016785, 940770, 949257, 1135983, 918598, 994...","[1037332, 856942, 6533936, 9297615, 856942, 55...","[856942, 1037332, 9297615, 6533936, 5577022, 8...","[856942, 839818, 9297615, 1007512, 5577022, 92...","[856942, 1037332, 9297615, 6533936, 5577022, 8..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 985999, 880888, 81...","[916122, 866211, 5569230, 933835, 1090931, 931...","[9802756, 961889, 965772, 963542, 841287, 1063...","[916122, 911974, 866211, 1076580, 5569230, 110...","[911974, 916122, 1076580, 866211, 1103898, 556...","[911974, 1137346, 1076580, 5569845, 1103898, 1...","[911974, 916122, 1076580, 866211, 1103898, 556..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[6391541, 1052294, 891423, 936470, 1137010, 11...","[1139525, 846550, 1128539, 865528, 6514160, 84...","[891423, 821741, 944588, 1082627, 1137346, 108...","[7467039, 862535, 5566809, 998519, 906202, 112...","[891423, 6391541, 821741, 1052294, 944588, 891...","[6391541, 891423, 1052294, 821741, 891423, 944...","[6391541, 1139525, 1052294, 846550, 891423, 11...","[6391541, 891423, 1052294, 821741, 891423, 944..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[13003092, 972416, 995598, 923600, 1138596, 10...","[948650, 5569845, 890739, 941361, 1074754, 110...","[1026118, 965267, 878996, 863632, 871611, 1084...","[7442505, 1113675, 948640, 907099, 1092669, 86...","[1026118, 13003092, 965267, 972416, 878996, 99...","[13003092, 1026118, 972416, 965267, 995598, 87...","[13003092, 948650, 972416, 5569845, 995598, 89...","[13003092, 1026118, 972416, 965267, 995598, 87..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[998519, 894360, 7147142, 9338009, 896666, 939...","[5585510, 1044078, 7147145, 12384779, 9707461,...","[912451, 1039627, 7142937, 1100140, 10285022, ...","[852015, 6552318, 8090550, 868888, 8069118, 80...","[912451, 998519, 1039627, 894360, 7142937, 714...","[998519, 912451, 894360, 1039627, 7147142, 714...","[998519, 5585510, 894360, 1044078, 7147142, 71...","[998519, 912451, 894360, 1039627, 7147142, 714..."


### Recall@50 of matching

In [ ]:
TOPK_RECALL = 50

In [ ]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.06525657038145165),
 ('als_and_own_rec', 0.06299332260890188),
 ('own_and_als_rec', 0.06299332260890188),
 ('als_and_sim_item_rec', 0.06299332260890188),
 ('own_and_sim_item_rec', 0.05722529081286704),
 ('als_rec', 0.048973301101330115),
 ('sim_item_rec', 0.03423574764392497),
 ('sim_user_rec', 0.007033025514520061)]

Лучший recall показали кандидаты, созданные по принципу own_recommendations.
Оценим метрику при различных k.

In [ ]:
{20: sorted(calc_recall(result_eval_matcher, 20), key=lambda x: x[1],reverse=True),
 50: sorted(calc_recall(result_eval_matcher, 50), key=lambda x: x[1],reverse=True),
 100: sorted(calc_recall(result_eval_matcher, 100), key=lambda x: x[1],reverse=True),
 200: sorted(calc_recall(result_eval_matcher, 200), key=lambda x: x[1],reverse=True),
 500: sorted(calc_recall(result_eval_matcher, 500), key=lambda x: x[1],reverse=True),
 }

{20: [('own_rec', 0.039284276793729055),
  ('als_and_own_rec', 0.0386479477757577),
  ('own_and_als_rec', 0.0386479477757577),
  ('als_and_sim_item_rec', 0.0386479477757577),
  ('own_and_sim_item_rec', 0.03393962363774048),
  ('als_rec', 0.029895932599008124),
  ('sim_item_rec', 0.017895038634712764),
  ('sim_user_rec', 0.004004676342633871)],
 50: [('own_rec', 0.06525657038145165),
  ('als_and_own_rec', 0.06299332260890188),
  ('own_and_als_rec', 0.06299332260890188),
  ('als_and_sim_item_rec', 0.06299332260890188),
  ('own_and_sim_item_rec', 0.05722529081286704),
  ('als_rec', 0.048973301101330115),
  ('sim_item_rec', 0.03423574764392497),
  ('sim_user_rec', 0.007033025514520061)],
 100: [('own_rec', 0.06525657038145165),
  ('als_and_own_rec', 0.06299332260890188),
  ('own_and_als_rec', 0.06299332260890188),
  ('als_and_sim_item_rec', 0.06299332260890188),
  ('own_and_sim_item_rec', 0.05722529081286704),
  ('als_rec', 0.048973301101330115),
  ('sim_item_rec', 0.03423574764392497),
  

С увеличением k метрика растет значительно медленнее. Оптимально подбирать 50-100 кандидатов, чтобы не нагружать модель.

### Precision@5 of matching

In [ ]:
sorted(calc_precision(result_eval_matcher, 5), key=lambda x: x[1],reverse=True)

[('own_rec', 0.17712691771269176),
 ('own_and_als_rec', 0.1684797768479777),
 ('als_and_sim_item_rec', 0.1684797768479777),
 ('als_and_own_rec', 0.15602045560204558),
 ('own_and_sim_item_rec', 0.1492329149232915),
 ('als_rec', 0.11464435146443515),
 ('sim_item_rec', 0.06527196652719666),
 ('sim_user_rec', 0.012831241283124128)]

## Задание 2

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

## Подготовка данных для трейна

In [ ]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [ ]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))


In [ ]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [ ]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [ ]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [ ]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1105426
0,2070,1097350
0,2070,879194
0,2070,948640


### Check warm start

In [ ]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (107550, 2) Users: 2151 Items: 4574


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [ ]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

In [ ]:
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


#### Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей

In [ ]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [ ]:
df_ranker_train.target.value_counts()

0.0    99177
1.0     7795
Name: target, dtype: int64

In [ ]:
df_ranker_train.head(5)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0
2,2070,879194,0.0
3,2070,948640,0.0
4,2070,928263,0.0


In [ ]:
df_ranker_train['target'].mean()

0.07286953595333358

## Подготавливаем фичи для обучения модели

In [ ]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [ ]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее кол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

In [ ]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [ ]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [ ]:
#Средний чек
sales_value = data[['user_id', 'sales_value']]
sales_value = sales_value.groupby('user_id')['sales_value'].sum().reset_index()
df_ranker_train = df_ranker_train.merge(sales_value, on='user_id', how='left')
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86


In [ ]:

#Средние покупки в месяц по пользователю (частота покупок в месяц)
by_per_week_on_user = data[['user_id', 'week_no', 'item_id']]
by_per_week_on_user = by_per_week_on_user.groupby(['user_id', 'week_no'])['item_id'].nunique().reset_index()
by_per_week_on_user.rename(columns={'item_id': 'buy_count_per_week_on_user'}, inplace=True)
by_per_week_on_user['month_no'] = (by_per_week_on_user['week_no']-1)//4+1
by_per_month_on_user = by_per_week_on_user.groupby(['user_id', 'month_no'])['buy_count_per_week_on_user'].sum().reset_index()
by_per_month_on_user.rename(columns={'buy_count_per_week_on_user': 'buy_per_month_on_user'}, inplace=True)
max_month = by_per_month_on_user['month_no'].max()
by_per_month_on_user = by_per_month_on_user.groupby('user_id')['buy_per_month_on_user'].sum().reset_index()
by_per_month_on_user['buy_per_month_on_user'] = by_per_month_on_user['buy_per_month_on_user']/max_month
by_per_month_on_user.rename(columns={'buy_per_month_on_user': 'average_month_buy_on_user'}, inplace=True)
by_per_month_on_user.head()

,user_id,average_month_buy_on_user
0,1,66.166667
1,2,27.208333
2,3,36.750000
3,4,12.416667
4,5,9.000000


In [ ]:
df_ranker_train = df_ranker_train.merge(by_per_month_on_user, on='user_id', how='left')
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667


In [ ]:
#Средние недельные покупки товара
item_buy_in_week = data.groupby(['item_id', 'week_no']).size().reset_index()
item_buy_in_week = item_buy_in_week.groupby('item_id')[0].mean().reset_index()
item_buy_in_week.rename(columns={0: 'average_item_buy_on_week'}, inplace=True)
df_ranker_train = df_ranker_train.merge(item_buy_in_week, on='item_id', how='left')
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user,average_item_buy_on_week
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.745763
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,90.416667,1.275000


In [ ]:
#Цена
data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
price = data.groupby('item_id')['price'].mean().reset_index()
df_ranker_train = df_ranker_train.merge(price, on='item_id', how='left')
list(df_ranker_train['department'].unique())

['DELI',
 'GROCERY',
 'DRUG GM',
 'MEAT-PCKGD',
 'SEAFOOD-PCKGD',
 'NUTRITION',
 'PRODUCE',
 'PASTRY',
 'MEAT',
 'SEAFOOD',
 'SPIRITS',
 'FLORAL',
 'COSMETICS',
 'MISC. TRANS.']

In [ ]:
data = data.merge(item_features[['item_id', 'department']], on='item_id', how='left')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,department
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39,PRODUCE
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,0.82,PRODUCE
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,0.99,PRODUCE
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,1.21,PRODUCE
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50,PRODUCE


In [ ]:
#Промежуточный датафрейм для вычисления средних покупок по категории по конкретным юзерам
buy_cat_reduce_average = data.groupby(['user_id', 'week_no', 'department'])['item_id'].count().reset_index()
buy_cat_reduce_average.rename(columns={'item_id': 'count_per_cat_on_user'}, inplace=True)
buy_cat_reduce_average = buy_cat_reduce_average.groupby(['user_id', 'department'])['count_per_cat_on_user'].mean().reset_index()

In [ ]:

#Промежуточный датафрейм для вычисления средних покупок по категории по всем юзерам
buy_cat_reduce_average_all_users = data.groupby(['department', 'week_no'])['item_id'].count().reset_index()
buy_cat_reduce_average_all_users.rename(columns={'item_id': 'count_per_cat'}, inplace=True)
buy_cat_reduce_average_all_users = buy_cat_reduce_average_all_users.groupby('department')['count_per_cat'].mean().reset_index()
buy_cat_reduce_average_all_users.head()

,department,count_per_cat
0,,75.894737
1,AUTOMOTIVE,1.404762
2,CHARITABLE CONT,2.000000
3,CHEF SHOPPE,9.865672
4,CNTRL/STORE SUP,1.750000


In [ ]:
buy_cat_reduce_average = buy_cat_reduce_average.merge(buy_cat_reduce_average_all_users, on='department')
buy_cat_reduce_average['average_subtraction_on_cat'] = buy_cat_reduce_average['count_per_cat_on_user'] - buy_cat_reduce_average['count_per_cat']
buy_cat_reduce_average['average_division_on_cat'] = buy_cat_reduce_average['count_per_cat_on_user']/buy_cat_reduce_average['count_per_cat']
buy_cat_reduce_average.head()
#Делая последние 2 фичи из списка, мы получаем еще 4 фичи, которые и замерджим в df_ranker_train

,user_id,department,count_per_cat_on_user,count_per_cat,average_subtraction_on_cat,average_division_on_cat
0,1,,1.090909,75.894737,-74.803828,0.014374
1,6,,1.000000,75.894737,-74.894737,0.013176
2,7,,1.000000,75.894737,-74.894737,0.013176
3,8,,1.142857,75.894737,-74.751880,0.015058
4,13,,1.400000,75.894737,-74.494737,0.018447


In [ ]:
df_ranker_train = df_ranker_train.merge(buy_cat_reduce_average, on=['user_id', 'department'])
df_ranker_train

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,household_size_desc,kid_category_desc,sales_value,average_month_buy_on_user,average_item_buy_on_week,price,count_per_cat_on_user,count_per_cat,average_subtraction_on_cat,average_division_on_cat
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.745763,3.796408,1.333333,611.484211,-610.150877,0.002180
1,2070,1016298,0.0,4074,DELI,National,CHICKEN/POULTRY,CHIX:VALUE ADDED (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.563636,3.101043,1.333333,611.484211,-610.150877,0.002180
2,2070,8015407,0.0,1216,DELI,National,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.971831,2.857000,1.333333,611.484211,-610.150877,0.002180
3,2070,1054185,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,1,None/Unknown,6981.86,90.416667,1.631579,3.797258,1.333333,611.484211,-610.150877,0.002180
4,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,...,1,None/Unknown,6981.86,90.416667,1.275000,11.460588,18.939024,15989.789474,-15970.850449,0.001184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105548,1745,1135566,0.0,69,PASTRY,Private,PIES,PIES: FRUIT/NUT,,45-54,...,2,None/Unknown,2417.12,35.916667,2.600000,5.172527,1.200000,371.326316,-370.126316,0.003232
105549,1745,8011291,0.0,69,PASTRY,Private,PIES,PIES: FRUIT/NUT,,45-54,...,2,None/Unknown,2417.12,35.916667,2.609756,5.254486,1.200000,371.326316,-370.126316,0.003232
105550,1745,1107039,0.0,1646,PRODUCE,National,SALAD MIX,GARDEN PLUS,18 OZ,45-54,...,2,None/Unknown,2417.12,35.916667,1.985714,3.397050,1.891892,2510.789474,-2508.897582,0.000754
105551,1745,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,...,2,None/Unknown,2417.12,35.916667,1.157895,6.596818,2.500000,2694.557895,-2692.057895,0.000928


In [ ]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [ ]:
# Выделим категориальные признаки 
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'sales_value',
 'average_month_buy_on_user',
 'average_item_buy_on_week',
 'price',
 'count_per_cat_on_user',
 'count_per_cat',
 'average_subtraction_on_cat',
 'average_division_on_cat']

In [ ]:
df_ranker_train.isna().sum()

user_id                           0
item_id                           0
target                            0
manufacturer                      0
department                        0
brand                             0
commodity_desc                    0
sub_commodity_desc                0
curr_size_of_product              0
age_desc                      66144
marital_status_code           66144
income_desc                   66144
homeowner_desc                66144
hh_comp_desc                  66144
household_size_desc           66144
kid_category_desc             66144
sales_value                       0
average_month_buy_on_user         0
average_item_buy_on_week          0
price                             0
count_per_cat_on_user             0
count_per_cat                     0
average_subtraction_on_cat        0
average_division_on_cat           0
dtype: int64

In [ ]:
#Чистим null, т.к null у категориальных переменных заменим их Unknown, просто как отдельная категория
df_ranker_train = df_ranker_train.fillna('Unknown')
df_ranker_train.isna().sum()

user_id                       0
item_id                       0
target                        0
manufacturer                  0
department                    0
brand                         0
commodity_desc                0
sub_commodity_desc            0
curr_size_of_product          0
age_desc                      0
marital_status_code           0
income_desc                   0
homeowner_desc                0
hh_comp_desc                  0
household_size_desc           0
kid_category_desc             0
sales_value                   0
average_month_buy_on_user     0
average_item_buy_on_week      0
price                         0
count_per_cat_on_user         0
count_per_cat                 0
average_subtraction_on_cat    0
average_division_on_cat       0
dtype: int64

## Обучение модели ранжирования

In [ ]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats,
                     n_jobs=-1,
#                      verbose=0
                   )

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

/home/aleks/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aleks/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aleks/.local/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [ ]:
df_ranker_predict = df_ranker_train.copy()

In [ ]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [ ]:
df_ranker_predict['proba_item_purchase'][:10]

0    0.010799
1    0.027081
2    0.017057
3    0.005540
4    0.015196
5    0.049814
6    0.073998
7    0.037001
8    0.072084
9    0.043393
Name: proba_item_purchase, dtype: float64

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [ ]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Eval matching on test dataset

In [ ]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 9.2 s, sys: 0 ns, total: 9.2 s
Wall time: 9.22 s


In [ ]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, 5), key=lambda x: x[1], reverse=True)

[('own_rec', 0.14441176470588235)]

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [ ]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [ ]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [ ]:
print(*sorted(calc_precision(result_eval_ranker, 5), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.16898172323759791)
('own_rec', 0.14441176470588235)


/home/aleks/rec sys/webinar_6/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


ВЫВОД: Двухуровневая модель дает ощутимый прирост качества. Для улучшения можно попробовать подобрать параметры модели. 